<a href="https://colab.research.google.com/github/alexontour/snippets/blob/main/snip_fhir_create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Author: Alexander Kollmann, 08/2022**

---

**Funktion**

FHIR-Server ansprechen (HAPI)

FHIR - Ressorcen schreiben und verknüpfen (versch. Möglichkeiten)

> Patient, Encounter, Observation, Condition

Bulk-Upload aus CSV-Datei

---

**Referenzen**

https://colab.research.google.com/drive/1OSuqqACnCqw8h67E7DKDMyIGO1C2qbEq?usp=sharing#scrollTo=rU-lfJob3fOf

https://github.com/Alvearie/FHIR-from-Jupyter

https://pypi.org/project/fhirclient/

https://github.com/intrahealth/client-registry/blob/master/docs/notebooks/load_bulk_data_in_python.ipynb

https://github.com/CODA-19/csv-to-fhir/blob/master/demo_csv-json.py 

https://github.com/Healthedata1/smart_pythonclient_example/blob/master/Argo-patient-example.ipynb

https://www.convertcsv.com/xml-to-csv.htm

---



In [1]:
import json
import requests
from collections import OrderedDict
from io import StringIO
from IPython.display import IFrame

In [2]:
# Base URL zum FHIR-Server
url = "http://hapi.fhir.org/baseR4/" # Open HAPI FHIR Server
#url = "https://server.fire.ly/R4/" # Open Firely Server
#url = "	http://localhost:8080/fhir"

# Header definieren
headers = {"Content-Type": "application/fhir+json;charset=utf-8"}

# **Resource Patient** 

https://www.hl7.org/fhir/patient.html

Demographics and other administrative information about an individual or animal receiving care or other health-related services. This Resource covers data about patients and animals involved in a wide range of health-related activities.

In [ ]:

# HL7 FHIR defines a set of "resources" for exchanging information.
IFrame('https://www.hl7.org/fhir/patient.html', width=1200, height=330)

In [3]:
# Option 1: JSON - Ressouce erstellen lt. Datenmodell
# json.load() takes a file object and returns the json object. It is used to read JSON encoded data from a file and convert it into a 
# Python dictionary and deserialize a file itself i.e. it accepts a file object.

patient = json.loads("""{
   "resourceType":"Patient",
   "name":[
      {
         "given": "Alex",
         "family": "MrXXX",
         "text":"Alex MrXXX",
         "use": "official"
      }
   ],
   "gender":"male",
   "birthDate":"1950-01-01"
}""")

In [4]:
# FHIR Ressource an FHIR-Server senden (POST)
response = requests.request("POST", url + "Patient", headers=headers, data=str(patient))

# Ergebnis (Response) verarbeiten
patient_id = json.loads(response.text)['id']
patient_name = json.loads(response.text)['name']
#response.json()

print("Patient-ID: " + patient_id)
#print(patient_name[0]['given'])

Patient-ID: 7168516


In [ ]:
# Option 2: Erzeugen der FHIR-Ressource als Python-Dictionary in eine Funktion auslagern
# Dictionaries are used to store data values in key:value pairs. A dictionary is a collection which is ordered*, changeable and do not allow duplicates.

def create_patient():

    body = {
        "name":[
            {
              "given": "Christiano",
              "family": "Ronaldo",
              "text":"Christiano Ronaldo",
              "use": "official"
            }
        ],
        "birthDate":"1950-01-01",
        "gender": "male",
        "resourceType": "Patient",
    }

    io = StringIO()
    json.dump(body, io, indent=2)

    return io


In [ ]:
# FHIR Ressource über Funktionsaufruf erstellen unn an FHIR-Server senden (POST)
response = requests.request("POST", url + "Patient", headers=headers, data=create_patient().getvalue())

patient_id = json.loads(response.text)['id']

print("Patient-ID: "+ patient_id)

Patient-ID: 7164858


# **Resource Encounter**

https://www.hl7.org/fhir/encounter.html

A patient encounter is further characterized by the setting in which it takes place. Amongst them are ambulatory, emergency, home health, inpatient and virtual encounters. An Encounter encompasses the lifecycle from pre-admission, the actual encounter (for ambulatory encounters), and admission, stay and discharge (for inpatient encounters). During the encounter the patient may move from practitioner to practitioner and location to location.

In [ ]:
def create_encounter(patient_id):

    body = {
        "status": "finished",
        "class": {
            "system": "http://hl7.org/fhir/v3/ActCode",
            "code": "IMP",
            "display": "inpatient encounter",
        },
        "reason": [
            {
                "text": "The patient had an abnormal heart rate. She was concerned about this."
            }
        ],
        "subject": {"reference": "Patient/{}".format(patient_id)},
        "resourceType": "Encounter",
    }

    io = StringIO()
    json.dump(body, io, indent=2)

    return io

response = requests.request("POST", url + "Encounter", headers=headers, data=create_encounter(patient_id).getvalue())

encounter_id = json.loads(response.text)['id']
#response.json()

print("Encounter-ID: " + encounter_id)

Encounter-ID: 7164859


# **Resource Observation**

https://www.hl7.org/fhir/observation.html

Measurements and simple assertions made about a patient, device or other subject. Observations are a central element in healthcare, used to support diagnosis, monitor progress, determine baselines and patterns and even capture demographic characteristics.

In [ ]:
# die Observation referenziert auf einen Patienten und einen Encounter

def create_observation(patient_id,encounter_id):

    body = {
        "resourceType": "Observation",
        "code": {
            "coding": [
                {
                    "system": "http://loinc.org",
                    "code": "8867-4",
                    "display": "Heart rate",
                }
            ]
        },
        "status": "final",
        "subject": {"reference": "Patient/{}".format(patient_id)},
        "effectiveDateTime": "2019-01-01T00:00:00+00:00",
        "valueQuantity": {"value": 80, "unit": "bpm"},
        "context": {"reference": "Encounter/{}".format(encounter_id)},
    }

    io = StringIO()
    json.dump(body, io, indent=2)

    return io

response = requests.request("POST", url + "Observation", headers=headers, data=create_observation(patient_id,encounter_id).getvalue())

observation_id = json.loads(response.text)['id']
#response.json()

print("Observation-ID: " + observation_id)

Observation-ID: 7164860


# **Resource Condition**

https://www.hl7.org/fhir/condition.html

A clinical condition, problem, diagnosis, or other event, situation, issue, or clinical concept that has risen to a level of concern. This resource is used to record detailed information about a condition, problem, diagnosis, or other event, situation, issue, or clinical concept that has risen to a level of concern.

In [ ]:
# Option 3: FHIR-Ressource über Orderd-Dictionary erstellen
# An OrderedDict is a dictionary subclass that remembers the order that keys were first inserted. 
# A regular dict doesn’t track the insertion order and iterating it gives the values in an arbitrary order. By contrast, the order the items are inserted is remembered by OrderedDict.

problem = OrderedDict()
problem['resourceType'] = 'Condition'
#problem['id'] = 'example'
problem['patient'] = {"reference": "Patient/{}".format(patient_id)}
#problem['encounter'] = {'reference': 'Encounter/67890'}
problem['dateRecorded'] = '2021-04-01T13:00'
problem['code'] = {'coding':
                       [{'system': 'http://hl7.org/fhir/sid/icd-10-us',
                         'code': 'E10.65',
                         'display': 'Type 1 Diabetes Mellitus with Hyperglycemia'}]}
problem['clinicalStatus'] = "active"
problem['verificationStatus'] = 'provisional'

io = StringIO()
json.dump(problem, io, indent=2)
#print(io.getvalue())

response = requests.request("POST", url + "Condition", headers=headers, data=io.getvalue())

condition_id = json.loads(response.text)['id']
#response.json()

print("Condition-ID: " + condition_id)

Condition-ID: 7164861


# Bulk - Upload

In [ ]:
!pip install fhirclient

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 683 kB 5.3 MB/s 
     |████████████████████████████████| 41 kB 238 kB/s 


In [ ]:
import fhirclient.models.patient as p
import fhirclient.models.humanname as hn
import fhirclient.models.contactpoint as cp
import fhirclient.models.fhirdate as fd
import fhirclient.models.identifier as ident
import fhirclient.models.observation as obs
import fhirclient.models.coding as co
import fhirclient.models.codeableconcept as coco
import fhirclient.models.quantity as qua
import fhirclient.models.fhirreference as ref

from fhirclient import client

import pandas as pd
import numpy as np

import time
import itertools


In [ ]:
# import patient-list
# Datei von Google Drive lesen (Wichtig: die Datei muss freigegeben werden)

doc_url='https://drive.google.com/file/d/1K9aeof3TFZYPPOKayZ-oQ_duza96_ZAX/view?usp=sharing'

file_id=doc_url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id
df = pd.read_csv(dwn_url, sep=';')

df = df.applymap(str)

# data cleaning
df['rec_id']= df['rec_id']
df['sex']= df['sex'].replace('f', 'female')
df['sex']= df['sex'].replace('m', 'male')

print('Number of records :', len(df))
print(df.head())

Number of records : 2
  rec_id     sex date_of_birth given_name surname
0      1    male    1967-12-07       Alex   Kolli
1      2  female    1959-03-04       Juli   Huber


In [ ]:
# import value-list
# Datei von Google Drive lesen (Wichtig: die Datei muss freigegeben werden)

doc_url='https://drive.google.com/file/d/1KCK-YqM3k_Zjkoa7o0Z30cEo1wXEXPds/view?usp=sharing'

file_id=doc_url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id
df_val = pd.read_csv(dwn_url, sep=',')

df_val = df_val.applymap(str)

# data cleaning
df_val.drop('DistanceMeters', inplace=True, axis=1)
df_val['HeartRateBpm'] = df_val['HeartRateBpm'].astype(float)

print('Number of records :', len(df_val))
print(df_val.head())

Number of records : 615
                       Time  HeartRateBpm
0  2022-11-24T16:51:38.000Z         106.0
1  2022-11-24T16:51:39.000Z         104.0
2  2022-11-24T16:51:40.000Z         105.0
3  2022-11-24T16:51:43.000Z         104.0
4  2022-11-24T16:51:44.000Z         106.0


In [ ]:
# https://hl7.org/fhir/observation-example-heart-rate.json.html
def create_observation_fhirclient(patient_id, ts, hr):
  coding = co.Coding()
  coding.system = "https://loinc.org"
  coding.code = "8867-4"
  coding.display = "Heart rate"
  code = coco.CodeableConcept()
  code.coding = [coding]
  code.text = "Heart rate"

  #Create a new observation using fhir.resources, we enter status and code inside the constructor since theuy are necessary to validate an observation
  observation = obs.Observation()
  observation.status = "final"
  observation.code = code
  #Set our category in our observation, category which hold codings which are composed of system, code and display
  coding = co.Coding()
  coding.system = "https://terminology.hl7.org/CodeSystem/observation-category"
  coding.code = "vital-signs"
  coding.display = "Vital Signs"
  category = coco.CodeableConcept()
  category.coding = [coding]
  observation.category = [category]

  #Set our effective date time in our observation
  observation.effectiveDateTime = fd.FHIRDate(ts)

  #Set our valueQuantity in our observation, valueQuantity which is made of a code, a unir, a system and a value
  valueQuantity = qua.Quantity()
  valueQuantity.code = "/min"
  valueQuantity.unit = "beats/minute"
  valueQuantity.system = "https://unitsofmeasure.org"
  valueQuantity.value = hr
  observation.valueQuantity = valueQuantity

  #Setting the reference to our patient using his id
  reference = ref.FHIRReference()
  reference.reference = f"Patient/{patient_id}"
  observation.subject = reference

  #print(json.dumps(observation.as_json(), indent=2))

  io = StringIO()
  json.dump(observation.as_json(), io, indent=2)

  #headers = {'Content-Type': 'application/json'}
  start = time.time()
  response = requests.request("POST", url + "Observation", headers=headers, data=io.getvalue())
  end = time.time()

  print(response)

  obs_id = json.loads(response.text)['id']
  print("Observation-ID: " + obs_id + " " + str(round((end - start), 1)) + " ms")

In [ ]:
limit = 100
for index, row in itertools.islice(df.iterrows(), limit):
  #for index, row in df.iterrows():
    patient = p.Patient() # not using rec_id as pandas id, leaving empty
    patient.gender = row['sex']
    name = hn.HumanName()
    name.given = [row['given_name']]
    name.family = row['surname']
    name.use = 'official'
    patient.name = [name]
    patient.birthDate = fd.FHIRDate(row['date_of_birth'])
    id = ident.Identifier()
    id.system = 'http://clientregistry.org/openmrs'
    id.value = row['rec_id']
    patient.identifier = [id]
    #print(json.dumps(patient.as_json(), indent=2))

    io = StringIO()
    json.dump(patient.as_json(), io, indent=2)

    #headers = {'Content-Type': 'application/json'}
    start = time.time()
    response = requests.request("POST", url + "Patient", headers=headers, data=io.getvalue())
    end = time.time()
    print(response)
    patient_id = json.loads(response.text)['id']
    print("Patient-ID: " + patient_id + " " + str(round((end - start), 1)) + " ms")
    
    # Funktion ausführen
    for index, row in df_val.iterrows():
      create_observation_fhirclient(patient_id, row['Time'], row['HeartRateBpm'])

<Response [201]>
Patient-ID: 7164862 0.3 ms
<Response [201]>
Observation-ID: 7164863 0.2 ms
<Response [201]>
Observation-ID: 7164864 0.1 ms
<Response [201]>
Observation-ID: 7164865 0.2 ms
<Response [201]>
Observation-ID: 7164866 0.1 ms
<Response [201]>
Observation-ID: 7164867 0.1 ms
<Response [201]>
Observation-ID: 7164868 0.6 ms
<Response [201]>
Observation-ID: 7164869 0.4 ms
<Response [201]>
Observation-ID: 7164870 0.6 ms
<Response [201]>
Observation-ID: 7164871 0.2 ms
<Response [201]>
Observation-ID: 7164872 0.1 ms
<Response [201]>
Observation-ID: 7164873 0.1 ms
<Response [201]>
Observation-ID: 7164874 0.1 ms
<Response [201]>
Observation-ID: 7164875 0.1 ms
<Response [201]>
Observation-ID: 7164876 0.1 ms
<Response [201]>
Observation-ID: 7164877 0.2 ms
<Response [201]>
Observation-ID: 7164878 0.2 ms
<Response [201]>
Observation-ID: 7164879 0.1 ms
<Response [201]>
Observation-ID: 7164880 0.1 ms
<Response [201]>
Observation-ID: 7164881 0.6 ms
<Response [201]>
Observation-ID: 7164882 0.2